In [115]:
import requests
import pandas as pd

# Replace 'your_api_key' with your actual FRED API key
api_key = '72271d1fc3590d9edc25838eab1aff0d'

# List of state series IDs (example, not complete)
state_series_ids = {
'Alabama': 'ALUR',
    'Alaska': 'AKUR',
    'Arizona': 'AZUR',
    'Arkansas': 'ARUR',
    'California': 'CAUR',
    'Colorado': 'COUR',
    'Connecticut': 'CTUR',
    'Delaware': 'DEUR',
    'Florida': 'FLUR',
    'Georgia': 'GAUR',
    'Hawaii': 'HIUR',
    'Idaho': 'IDUR',
    'Illinois': 'ILUR',
    'Indiana': 'INUR',
    'Iowa': 'IAUR',
    'Kansas': 'KSUR',
    'Kentucky': 'KYUR',
    'Louisiana': 'LAUR',
    'Maine': 'MEUR',
    'Maryland': 'MDUR',
    'Massachusetts': 'MAUR',
    'Michigan': 'MIUR',
    'Minnesota': 'MNUR',
    'Mississippi': 'MSUR',
    'Missouri': 'MOUR',
    'Montana': 'MTUR',
    'Nebraska': 'NEUR',
    'Nevada': 'NVUR',
    'New Hampshire': 'NHUR',
    'New Jersey': 'NJUR',
    'New Mexico': 'NMUR',
    'New York': 'NYUR',
    'North Carolina': 'NCUR',
    'North Dakota': 'NDUR',
    'Ohio': 'OHUR',
    'Oklahoma': 'OKUR',
    'Oregon': 'ORUR',
    'Pennsylvania': 'PAUR',
    'Rhode Island': 'RIUR',
    'South Carolina': 'SCUR',
    'South Dakota': 'SDUR',
    'Tennessee': 'TNUR',
    'Texas': 'TXUR',
    'Utah': 'UTUR',
    'Vermont': 'VTUR',
    'Virginia': 'VAUR',
    'Washington': 'WAUR',
    'West Virginia': 'WVUR',
    'Wisconsin': 'WIUR',
    'Wyoming': 'WYUR'
}

# Base URL for the FRED API
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Function to fetch yearly unemployment data for a given state
def fetch_unemployment_data(series_id, api_key):
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'frequency': 'm'  # 'a' for annual
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data['observations']

# Fetch data for all states
unemployment_data = {}
for state, series_id in state_series_ids.items():
    unemployment_data[state] = fetch_unemployment_data(series_id, api_key)

# Convert the data to a DataFrame for easier analysis
df_list = []
for state, observations in unemployment_data.items():
    state_df = pd.DataFrame(observations)
    state_df['state'] = state
    state_df['urr'] = pd.to_numeric(state_df['value'])  # Convert values to numeric
    state_df = state_df[['date', 'urr', 'state']]  # Select relevant columns
    df_list.append(state_df)

all_states_df = pd.concat(df_list, ignore_index=True)

# Display the DataFrame
print(all_states_df)

# Save the DataFrame to a CSV file
all_states_df.to_csv('unemployment_by_state_by_year.csv', index=False)


             date  urr    state
0      1976-01-01  6.6  Alabama
1      1976-02-01  6.6  Alabama
2      1976-03-01  6.6  Alabama
3      1976-04-01  6.5  Alabama
4      1976-05-01  6.4  Alabama
...           ...  ...      ...
28945  2023-11-01  2.9  Wyoming
28946  2023-12-01  2.9  Wyoming
28947  2024-01-01  2.8  Wyoming
28948  2024-02-01  2.8  Wyoming
28949  2024-03-01  2.8  Wyoming

[28950 rows x 3 columns]


In [117]:
all_states_df
all_states_df['date'] = pd.to_datetime(all_states_df['date'])
all_states_df['year'] = all_states_df['date'].dt.year
all_states_df = all_states_df.loc[all_states_df.year >= 2008]
all_states_df['urr'] = pd.to_numeric(all_states_df['urr'])
all_states_df_year = all_states_df.groupby(['year', 'state'])['urr'].mean().reset_index()

In [62]:
api_key = '72271d1fc3590d9edc25838eab1aff0d'
base_url = 'https://api.stlouisfed.org/fred/series/observations'

In [133]:
all_states_df_year.sort_values(by=['state','year'])
all_states_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/urr.csv')

In [120]:
def fetch_data(series_id,api_key, measure):
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'frequency': 'm'  # 'a' for annual
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df = df.loc[df.year >= 2008]
    index_to_drop  = df[df.value == '.'].index
    df.drop(index_to_drop, inplace=True)
    df[measure] = pd.to_numeric(df['value'])
    df_year = df.groupby(['year'])[measure].mean().reset_index()
    return df_year

In [121]:
consumer_sentiment_series = 'UMCSENT'
consumer_sentiment_df_year = fetch_data(consumer_sentiment_series, api_key, 'cons_sent')

In [165]:
consumer_sentiment_df_year.head()
consumer_sentiment_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/consumer_sentiment.csv')

In [123]:
cpi_median = 'MEDCPIM158SFRBCLE'
cpi_median_df_year = fetch_data(cpi_median, api_key, 'cpi')

In [164]:
cpi_median_df_year.head()
cpi_median_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/cpi.csv')

In [162]:
volatility_index = 'VIXCLS'
volatility_df_year = fetch_data(volatility_index, api_key, 'volatility')

In [163]:
volatility_df_year
volatility_df_year['pct_change'] = volatility_df_year.volatility.pct_change()
volatility_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/volatility.csv')

In [129]:
nasdaq_index = 'NASDAQCOM'
nasdaq_df_year = fetch_data(nasdaq_index, api_key, 'NASDAQ')

In [154]:
nasdaq_df_year['pct_change'] = nasdaq_df_year.NASDAQ.pct_change()
nasdaq_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/nasdaq.csv')

,year,NASDAQ,pct_change
0,2008,2162.420500,NaN
1,2009,1841.029000,-0.148626
2,2010,2347.699583,0.275211
3,2011,2680.418417,0.141721
4,2012,2965.767583,0.106457
5,2013,3537.690333,0.192841
6,2014,4374.313833,0.236489
7,2015,4943.493083,0.130119
8,2016,4982.490583,0.007889
9,2017,6231.280833,0.250636


In [157]:
all_states_df_year = all_states_df_year.sort_values(['state','year']).reset_index()

In [161]:
all_states_df_year.to_csv('C:/Users/appar/OneDrive/jup_nb/unemployment.csv')